In [1]:
print("pro")

pro


In [2]:
import cesium

In [3]:
from cesium import datasets

In [4]:
import numpy

In [5]:
eeg = datasets.fetch_andrzejak()


Loaded data from cached archive.


In [6]:
import scipy


In [7]:
eeg["classes"] = eeg["classes"].astype("U16")

In [8]:
import pywt

In [9]:
n_channels=7
eeg["dwts"] = [pywt.wavedec(m, pywt.Wavelet("db2"), level=n_channels-1)
               for m in eeg["measurements"]]

In [10]:
len(eeg["dwts"][0][6])

2050

In [11]:
from sklearn.cluster import KMeans

In [12]:
data = []
for x in range(0, 500):
    dp = [0,1,2,3,4,5,6]
    ip=[]
    for i in dp:
        index = KMeans(n_clusters=8, random_state=0).fit_predict((eeg["dwts"][x][i]).reshape(-1,1))
        index1 = 0
        index2 = 0
        index3 = 0
        index4 = 0
        index5 = 0
        index6 = 0
        index7 = 0
        index8 = 0
        length = len(index)
        for c in index:
            if(c==0):
                index1+=1
            if(c==1):
                index2+=1
            if(c==2):
                index3+=1
            if(c==3):
                index4+=1
            if(c==4):
                index5+=1
            if(c==5):
                index6+=1
            if(c==6):
                index7+=1
            if(c==7):
                index8+=1

        ip.append(index1/length)
        ip.append(index2/length)
        ip.append(index3/length) 
        ip.append(index4/length) 
        ip.append(index5/length) 
        ip.append(index6/length)
        ip.append(index7/length)
        ip.append(index8/length)
    data.append(ip)
len(data)

500

In [13]:
print(ip)

[0.13636363636363635, 0.19696969696969696, 0.22727272727272727, 0.18181818181818182, 0.09090909090909091, 0.045454545454545456, 0.10606060606060606, 0.015151515151515152, 0.12121212121212122, 0.2727272727272727, 0.12121212121212122, 0.10606060606060606, 0.12121212121212122, 0.015151515151515152, 0.15151515151515152, 0.09090909090909091, 0.15384615384615385, 0.11538461538461539, 0.17692307692307693, 0.1, 0.05384615384615385, 0.15384615384615385, 0.1076923076923077, 0.13846153846153847, 0.2713178294573643, 0.031007751937984496, 0.17829457364341086, 0.12015503875968993, 0.05426356589147287, 0.16279069767441862, 0.12790697674418605, 0.05426356589147287, 0.3346303501945525, 0.04085603112840467, 0.11673151750972763, 0.07003891050583658, 0.1614785992217899, 0.038910505836575876, 0.07782101167315175, 0.15953307392996108, 0.42592592592592593, 0.03898635477582846, 0.05458089668615984, 0.1442495126705653, 0.1530214424951267, 0.017543859649122806, 0.10721247563352826, 0.05847953216374269, 0.458048

In [14]:
len(index)

2050

In [15]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import optimizers

Using TensorFlow backend.


In [16]:
model = Sequential()
model.add(Dense(5, input_dim=56, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
rmsprop = optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=None, decay=0.0)
model.compile(optimizer=rmsprop,
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [17]:
result=[]
for x in range(0, 200):
    result.append(0)
for x in range(200, 500):
    result.append(1)
len(result)

500

In [18]:
result[499]
len(data[499])

56

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
X_train, X_test, y_train, y_test = train_test_split(data, result,stratify=result,test_size=0.4)
#print (X_train.len())
#print (y_train.shape)
#print (X_test.shape)
#print (y_test.shape)
print(numpy.shape(X_train))
print(numpy.shape(y_train))
print(numpy.shape(X_test))
print(numpy.shape(y_test))

(300, 56)
(300,)
(200, 56)
(200,)


In [21]:
model.fit(numpy.array(X_train), numpy.array(y_train),
          epochs=50,batch_size=1)
score = model.evaluate(numpy.array(X_test),numpy.array(y_test), batch_size=1)


Epoch 1/50
300/300 [==============================] - 1s 4ms/step - loss: 0.7050 - acc: 0.4733
Epoch 2/50
300/300 [==============================] - 0s 1ms/step - loss: 0.6933 - acc: 0.4800
Epoch 3/50
300/300 [==============================] - 0s 1ms/step - loss: 0.6862 - acc: 0.5333
Epoch 4/50
300/300 [==============================] - 0s 1ms/step - loss: 0.6822 - acc: 0.5933
Epoch 5/50
300/300 [==============================] - 0s 1ms/step - loss: 0.6799 - acc: 0.5967
Epoch 6/50
300/300 [==============================] - 0s 1ms/step - loss: 0.6783 - acc: 0.5933
Epoch 7/50
300/300 [==============================] - 0s 1ms/step - loss: 0.6774 - acc: 0.5967
Epoch 8/50
300/300 [==============================] - 0s 1ms/step - loss: 0.6765 - acc: 0.5967
Epoch 9/50
300/300 [==============================] - 0s 1ms/step - loss: 0.6754 - acc: 0.5967
Epoch 10/50
300/300 [==============================] - 1s 2ms/step - loss: 0.6751 - acc: 0.5967
Epoch 11/50
300/300 [============================

In [22]:
score

[0.6646247699856758, 0.6]

In [23]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 5)                 285       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 291
Trainable params: 291
Non-trainable params: 0
_________________________________________________________________


In [24]:
from cesium import featurize
features_to_use = ["amplitude",
                   "percent_beyond_1_std",
                   "maximum",
                   "max_slope",
                   "median",
                   "median_absolute_deviation",
                   "percent_close_to_median",
                   "minimum",
                   "skew",
                   "std",
                   "weighted_average"]
fset_cesium = featurize.featurize_time_series(times=eeg["times"],
                                              values=eeg["measurements"],
                                              errors=None,
                                              features_to_use=features_to_use)
print(fset_cesium.head())

feature amplitude percent_beyond_1_std maximum     max_slope median  \
channel         0                    0       0             0      0   
0           143.5             0.327313   141.0  11107.796610   -4.0   
1           211.5             0.290212   169.0  20653.559322  -51.0   
2           165.0             0.302660   184.0  13537.627119   13.0   
3           171.5             0.300952   162.0  17008.813559   -4.0   
4           170.0             0.305101   152.0  13016.949153  -18.0   

feature median_absolute_deviation percent_close_to_median minimum      skew  \
channel                         0                       0       0         0   
0                            28.0                0.505004  -146.0  0.032805   
1                            32.0                0.640469  -254.0 -0.092715   
2                            31.0                0.515987  -146.0 -0.004100   
3                            31.0                0.541128  -181.0  0.063678   
4                           

In [25]:
import numpy as np
import scipy.stats

def mean_signal(t, m, e):
    return np.mean(m)

def std_signal(t, m, e):
    return np.std(m)

def mean_square_signal(t, m, e):
    return np.mean(m ** 2)

def abs_diffs_signal(t, m, e):
    return np.sum(np.abs(np.diff(m)))

def skew_signal(t, m, e):
    return scipy.stats.skew(m)

In [26]:
guo_features = {
    "mean": mean_signal,
    "std": std_signal,
    "mean2": mean_square_signal,
    "abs_diffs": abs_diffs_signal,
    "skew": skew_signal
}

fset_guo = featurize.featurize_time_series(times=eeg["times"], values=eeg["measurements"],
                                           errors=None,
                                           features_to_use=list(guo_features.keys()),
                                           custom_functions=guo_features)
print(fset_guo.head())

feature        mean2       mean      skew        std abs_diffs
channel            0          0         0          0         0
0        1650.122773  -4.132048  0.032805  40.411000   46948.0
1        5133.124725 -52.444716 -0.092715  48.812668   61118.0
2        2384.051989  12.705150 -0.004100  47.144789   51269.0
3        2231.742495  -3.992433  0.063678  47.072316   75014.0
4        2340.967781 -17.999268  0.142753  44.910958   52873.0


In [27]:
fset_dwt = featurize.featurize_time_series(times=None, values=eeg["dwts"], errors=None,
                                           features_to_use=list(guo_features.keys()),
                                           custom_functions=guo_features)
print(fset_dwt.head())

feature          mean2                                                        \
channel              0             1             2            3            4   
0         30062.870852  10207.929373   5825.967002  5546.670935  3216.802624   
1        200911.602497  16645.124399   8267.327747  9876.764535  5063.609754   
2         44413.838670  17445.284632  10473.428690  8024.522749  3651.422455   
3         20826.350863  11996.527751   7296.442497  8671.623599  4904.475281   
4         56868.658032  13238.772970   5921.599640  7144.565974  3375.749935   

feature                                 mean                        \
channel            5           6           0          1          2   
0         425.874858   33.453288  -23.689623  17.272795   6.816418   
1         821.614770   61.088168 -421.544159   5.119248 -11.762066   
2         500.386700   38.858553   93.055745 -26.823275  -2.348890   
3        1143.624462  130.582847  -38.543541  17.316768  -8.020088   
4         555.34774

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

train, test = train_test_split(np.arange(len(eeg["classes"])), random_state=0)

model_cesium = RandomForestClassifier(n_estimators=128, max_features="auto",
                                      random_state=0)
model_cesium.fit(fset_cesium.iloc[train], eeg["classes"][train])

model_guo = KNeighborsClassifier(3)
model_guo.fit(fset_guo.iloc[train], eeg["classes"][train])

model_dwt = KNeighborsClassifier(3)
model_dwt.fit(fset_dwt.iloc[train], eeg["classes"][train])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [29]:
from sklearn.metrics import accuracy_score

preds_cesium = model_cesium.predict(fset_cesium)
preds_guo = model_guo.predict(fset_guo)
preds_dwt = model_dwt.predict(fset_dwt)

print("Built-in cesium features: training accuracy={:.2%}, test accuracy={:.2%}".format(
          accuracy_score(preds_cesium[train], eeg["classes"][train]),
          accuracy_score(preds_cesium[test], eeg["classes"][test])))
print("Guo et al. features: training accuracy={:.2%}, test accuracy={:.2%}".format(
          accuracy_score(preds_guo[train], eeg["classes"][train]),
          accuracy_score(preds_guo[test], eeg["classes"][test])))
print("Wavelet transform features: training accuracy={:.2%}, test accuracy={:.2%}".format(
          accuracy_score(preds_dwt[train], eeg["classes"][train]),
          accuracy_score(preds_dwt[test], eeg["classes"][test])))

Built-in cesium features: training accuracy=100.00%, test accuracy=71.20%
Guo et al. features: training accuracy=83.47%, test accuracy=64.80%
Wavelet transform features: training accuracy=88.27%, test accuracy=79.20%


In [30]:
type(fset_dwt)

pandas.core.frame.DataFrame

In [31]:
'''put new feature of dwt subbands into a NN from hereon'''
data1  = fset_dwt.values
print(data1)

[[  30062.8708517    10207.92937265    5825.9670023  ...   41041.50056515
    29713.78613278   13817.93914032]
 [ 200911.60249697   16645.12439893    8267.3277472  ...   49078.7204449
    40493.54783627   17837.91095994]
 [  44413.83867006   17445.28463219   10473.42869033 ...   43031.29463546
    31717.33136014   14122.51402848]
 ...
 [ 215642.85196534  630264.26919028  709263.46518451 ...  408015.3528465
   347128.22454911  113080.09030677]
 [1968246.72666235 1510692.65061669 1628879.2886554  ...  481164.27251544
   431420.69959783  142841.83116537]
 [ 613778.63514278 1450463.84501033 2053943.01794631 ...  453838.30799646
   318848.61652704  100006.41631295]]


In [32]:
len(eeg["classes"])

500

In [33]:
result1=[]
for x in range(0, 200):
    result1.append(0)
for x in range(200, 500):
    result1.append(1)
len(result1)

500

In [34]:
model1 = Sequential()
model1.add(Dense(5, input_dim=35, activation='tanh'))
model1.add(Dense(1, activation='sigmoid'))
rmsprop1 = optimizers.RMSprop(lr=0.05, rho=0.9, epsilon=None, decay=0.0)
model1.compile(optimizer=rmsprop1,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [35]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(data1, result1,stratify=result1,test_size=0.4)
print(numpy.shape(X_train1))
print(numpy.shape(y_train1))
print(numpy.shape(X_test1))
print(numpy.shape(y_test1))

(300, 35)
(300,)
(200, 35)
(200,)


In [36]:
model1.fit(numpy.array(X_train1), numpy.array(y_train1),
          epochs=60,batch_size=1)
score1 = model1.evaluate(numpy.array(X_test1),numpy.array(y_test1), batch_size=1)
score1

Epoch 1/60
300/300 [==============================] - 1s 2ms/step - loss: 0.5151 - acc: 0.6633
Epoch 2/60
300/300 [==============================] - 0s 1ms/step - loss: 0.4403 - acc: 0.7233
Epoch 3/60
300/300 [==============================] - 0s 1ms/step - loss: 0.4582 - acc: 0.6933
Epoch 4/60
300/300 [==============================] - 0s 1ms/step - loss: 0.4444 - acc: 0.7433
Epoch 5/60
300/300 [==============================] - 0s 1ms/step - loss: 0.4457 - acc: 0.7067
Epoch 6/60
300/300 [==============================] - 0s 1ms/step - loss: 0.4467 - acc: 0.6933
Epoch 7/60
300/300 [==============================] - 0s 1ms/step - loss: 0.4393 - acc: 0.7167
Epoch 8/60
300/300 [==============================] - 0s 1ms/step - loss: 0.4390 - acc: 0.7167
Epoch 9/60
300/300 [==============================] - 1s 2ms/step - loss: 0.4324 - acc: 0.7233
Epoch 10/60
300/300 [==============================] - 1s 2ms/step - loss: 0.4348 - acc: 0.7267
Epoch 11/60
300/300 [============================

[0.440992369193683, 0.705]

In [37]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(data1, result1,stratify=result1,test_size=0.4)


In [38]:
#model2 with additional hidden layers, sigmoid in 2nd one gives better accuracy
model2 = Sequential()
model2.add(Dense(20, input_dim=35, activation='tanh'))
model2.add(Dense(10, activation='sigmoid'))
model2.add(Dense(5, activation='sigmoid'))
model2.add(Dense(3, activation='sigmoid'))
model2.add(Dense(1, activation='sigmoid'))
rmsprop2 = optimizers.RMSprop(lr=0.002, rho=0.9, epsilon=None, decay=0.0)
model2.compile(optimizer=rmsprop2,
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [39]:
model2.fit(numpy.array(X_train2), numpy.array(y_train2),
          epochs=50,batch_size=1)
score2 = model2.evaluate(numpy.array(X_test2),numpy.array(y_test2), batch_size=1)
score2

Epoch 1/50
300/300 [==============================] - 1s 2ms/step - loss: 0.6485 - acc: 0.6000
Epoch 2/50
300/300 [==============================] - 0s 1ms/step - loss: 0.5675 - acc: 0.6867
Epoch 3/50
300/300 [==============================] - 1s 2ms/step - loss: 0.4758 - acc: 0.8200
Epoch 4/50
300/300 [==============================] - 0s 1ms/step - loss: 0.4030 - acc: 0.8633
Epoch 5/50
300/300 [==============================] - 0s 1ms/step - loss: 0.3562 - acc: 0.8700
Epoch 6/50
300/300 [==============================] - 0s 1ms/step - loss: 0.3286 - acc: 0.8667
Epoch 7/50
300/300 [==============================] - 0s 2ms/step - loss: 0.3102 - acc: 0.8533
Epoch 8/50
300/300 [==============================] - 0s 2ms/step - loss: 0.2978 - acc: 0.8633
Epoch 9/50
300/300 [==============================] - 0s 2ms/step - loss: 0.2842 - acc: 0.8767
Epoch 10/50
300/300 [==============================] - 0s 1ms/step - loss: 0.2801 - acc: 0.8733
Epoch 11/50
300/300 [============================

[0.26801480679539963, 0.845]

In [40]:
#knn for binary classification
model_dwt1 = KNeighborsClassifier(2)
train, test = train_test_split(np.arange(len(eeg["classes"])), random_state=0)
model_dwt1.fit(fset_dwt.iloc[train], eeg["classes"][train])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=2, p=2,
           weights='uniform')

In [41]:
preds_dwt1 = model_dwt1.predict(fset_dwt)
print("Wavelet transform features: training accuracy={:.2%}, test accuracy={:.2%}".format(
          accuracy_score(preds_dwt1[train], eeg["classes"][train]),
          accuracy_score(preds_dwt1[test], eeg["classes"][test])))


Wavelet transform features: training accuracy=90.93%, test accuracy=74.40%


In [42]:
#knn for binary epileptic somehow better than above always 95+ acc.
total = 0
for x in range(0, 2000):
    X_train3, X_test3, y_train3,y_test3 = train_test_split(data1, result1,stratify=result1,test_size=0.4)
    model_dwt2 = KNeighborsClassifier(2)
    model_dwt2.fit(X_train3,y_train3)
    score3 = accuracy_score(y_test3,model_dwt2.predict(X_test3))
    total+=score3

In [43]:
score_avg = total/2000
score_avg

0.9585999999999993